In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import shutil

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_reduced = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
stop_times_original = pd.read_csv(gtfs_dir + 'stop_times.txt')

In [4]:
stop_times_reduced

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [5]:
stop_times = stop_times_reduced.merge(stop_times_original[['trip_id', 'stop_id', 'stop_sequence','arrival_time', 'departure_time']],
                                      how = 'left')

In [6]:
stop_times['INT_ID'] = stop_times['INT_ID'].astype(int)
stop_times = stop_times.drop(columns = ['stop_id', 'hr', 'stop_sequence', 'min'])
stop_times = stop_times.rename(columns = {'stop_sequence_new':'stop_sequence', 'INT_ID':'stop_id'})
stop_times

,trip_id,stop_sequence,stop_id,arrival_time,departure_time
0,32828689,1,13466640,5:15:00,5:15:00
1,32828689,2,13466531,5:15:43,5:15:43
2,32828689,3,13466677,5:16:26,5:16:26
3,32828689,4,13466754,5:17:08,5:17:08
4,32828689,5,13466699,5:18:00,5:18:00
...,...,...,...,...,...
3959705,32970576,7,13461602,25:56:08,25:56:08
3959706,32970576,8,13461912,25:56:44,25:56:44
3959707,32970576,9,13462284,25:57:21,25:57:21
3959708,32970576,10,13462427,25:57:58,25:57:58


In [7]:
stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence']]

In [8]:
trips = pd.read_csv(gtfs_dir + 'trips_weekday.csv')
routes = trips.copy()

In [9]:
trips = trips[['route_short_name','trip_id', 'service_id', 'trip_headsign', 'direction_id']]
trips = trips.rename(columns = {'route_short_name':'route_id'})

In [10]:
trips = trips[['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id']]

In [11]:
routes_original = pd.read_csv(gtfs_dir + 'routes.txt')
routes_original

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,47526,1,1,LINE 1 (YONGE-UNIVERSITY),NaN,1,NaN,D5C82B,000000
1,47527,1,10,VAN HORNE,NaN,3,NaN,804000,FFFFFF
2,47528,1,100,FLEMINGDON PARK,NaN,3,NaN,FF00FF,FFFFFF
3,47529,1,101,DOWNSVIEW PARK,NaN,3,NaN,00FF00,FFFFFF
4,47530,1,102,MARKHAM RD,NaN,3,NaN,00FF00,FFFFFF
...,...,...,...,...,...,...,...,...,...
194,47719,1,95,YORK MILLS,NaN,3,NaN,FF8000,FFFFFF
195,47720,1,96,WILSON,NaN,3,NaN,804000,FFFFFF
196,47721,1,97,YONGE,NaN,3,NaN,804000,FFFFFF
197,47722,1,98,WILLOWDALE - SENLAC,NaN,3,NaN,00FF00,FFFFFF


In [12]:
routes = routes[['route_id', 'route_short_name', 'route_long_name']].drop_duplicates().merge(
    routes_original[['route_id', 'agency_id', 'route_type']], how = 'left')
routes = routes.drop(columns = ['route_id'])
routes['route_id'] = routes['route_short_name']

In [13]:
routes['route_long_name'] = np.where(routes['route_id'] > 1000, routes['route_long_name'] + ' ROCKET', routes['route_long_name'])

In [14]:
routes = routes[['route_id', 'agency_id', 'route_short_name', 'route_long_name', 'route_type']]
routes

,route_id,agency_id,route_short_name,route_long_name,route_type
0,1,1,1,LINE 1 (YONGE-UNIVERSITY),1
1,10,1,10,VAN HORNE,3
2,100,1,100,FLEMINGDON PARK,3
3,101,1,101,DOWNSVIEW PARK,3
4,102,1,102,MARKHAM RD,3
...,...,...,...,...,...
195,96,1,96,WILSON,3
196,97,1,97,YONGE,3
197,98,1,98,WILLOWDALE - SENLAC,3
198,99,1,99,ARROW ROAD,3


In [15]:
stops = pd.read_csv(simplify_dir + 'int_stop.csv')
intersections = gpd.read_file('GIS/intersection-file-wgs84/CENTRELINE_INTERSECTION_WGS84.shp')
stations = pd.read_csv('GIS/stations.csv')

In [16]:
intersections

,INT_ID,ELEV_ID,INTERSEC5,CLASSIFI6,CLASSIFI7,NUM_ELEV,ELEVATIO9,ELEVATIO10,ELEV_LEVEL,ELEVATION,ELEVATIO13,HEIGHT_R14,HEIGHT_R15,X,Y,LONGITUDE,LATITUDE,OBJECTID,geometry
0,13469938,20548,GARDINER EXPRESS / Kipling Ave,SEUML,Pseudo Intersection-Overpass/Underpass,2,509200,Pseudo,0,0.0,None,0.0,None,302703.415,4830674.977,-79.525779,43.618007,41.0,POINT (-79.52578 43.61801)
1,13469938,29774,GARDINER EXPRESS / Kipling Ave,SEUML,Pseudo Intersection-Overpass/Underpass,2,509200,Pseudo,1,0.0,None,0.0,None,302703.415,4830674.977,-79.525779,43.618007,42.0,POINT (-79.52578 43.61801)
2,13469952,9472,Kipling Ave / Kipling S Gardiner C E Ramp,MNRML,Minor-Multi Level,2,501300,Minor,1,0.0,None,0.0,None,302713.398,4830626.325,-79.525656,43.617569,39.0,POINT (-79.52566 43.61757)
3,13469952,28079,Kipling Ave / Kipling S Gardiner C E Ramp,MNRML,Minor-Multi Level,2,509200,Pseudo,0,0.0,None,0.0,None,302713.398,4830626.325,-79.525656,43.617569,40.0,POINT (-79.52566 43.61757)
4,13469946,32748,GARDINER EXPRESS / Kipling Ave,SEUML,Pseudo Intersection-Overpass/Underpass,2,509200,Pseudo,0,0.0,None,0.0,None,302711.225,4830637.839,-79.525683,43.617673,45.0,POINT (-79.52568 43.61767)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53730,13455527,19789,Divadale Dr / Don Avon Dr,MNRSL,Minor-Single Level,1,501300,Minor,0,0.0,None,0.0,None,315822.001,4841554.000,-79.363021,43.715855,12134.0,POINT (-79.36302 43.71585)
53731,13455391,7897,Broadway Ave / Don Avon Dr,MNRSL,Minor-Single Level,1,501300,Minor,0,0.0,None,0.0,None,315793.094,4841650.500,-79.363378,43.716724,12135.0,POINT (-79.36338 43.71672)
53732,13455352,21959,Brentcliffe Rd / Broadway Ave / Rykert Cres,MNRSL,Minor-Single Level,1,501300,Minor,0,0.0,None,0.0,None,315875.688,4841678.000,-79.362352,43.716970,12138.0,POINT (-79.36235 43.71697)
53733,13455357,13229,Brentcliffe Rd,MNRSL,Minor-Single Level,1,501300,Minor,0,0.0,None,0.0,None,315966.219,4841673.500,-79.361229,43.716928,12136.0,POINT (-79.36123 43.71693)


In [17]:
stops = stops.merge(intersections[['INT_ID', 'INTERSEC5']], how = 'left')
stops = stops.merge(stations, how = 'left')

In [18]:
stops

,INT_ID,int_lon,int_lat,INTERSEC5,name,latitude,longitude
0,110,-79.396989,43.697748,NaN,DAVISVILLE STATION,43.697748,-79.396989
1,205,-79.475991,43.652048,NaN,RUNNYMEDE STATION,43.652048,-79.475991
2,214,-79.403889,43.667249,NaN,SPADINA STATION,43.667249,-79.403889
3,208,-79.452723,43.657023,NaN,DUNDAS WEST STATION,43.657023,-79.452723
4,115,-79.412755,43.767867,NaN,NORTH YORK CENTRE STATION,43.767867,-79.412755
...,...,...,...,...,...,...,...
3666,13459088,-79.273038,43.692872,Warden Ave / Hollis Ave,NaN,NaN,NaN
3667,13456297,-79.265758,43.709715,Danforth Rd / North Edgely Ave,NaN,NaN,NaN
3668,13456297,-79.265758,43.709715,Danforth Rd / North Edgely Ave,NaN,NaN,NaN
3669,13441958,-79.175723,43.818247,Zoo Rd / Kirkhams Rd / Kirkhams Rd Zoo Rd Ramp,NaN,NaN,NaN


In [19]:
stops['stop_name'] = np.where(stops['INTERSEC5'].isna(), stops['name'], stops['INTERSEC5'])

In [20]:
stops = stops.rename(columns = {'int_lon':'stop_lon', 'int_lat':'stop_lat', 'INT_ID':'stop_id'})
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stops

,stop_id,stop_name,stop_lat,stop_lon
0,110,DAVISVILLE STATION,43.697748,-79.396989
1,205,RUNNYMEDE STATION,43.652048,-79.475991
2,214,SPADINA STATION,43.667249,-79.403889
3,208,DUNDAS WEST STATION,43.657023,-79.452723
4,115,NORTH YORK CENTRE STATION,43.767867,-79.412755
...,...,...,...,...
3666,13459088,Warden Ave / Hollis Ave,43.692872,-79.273038
3667,13456297,Danforth Rd / North Edgely Ave,43.709715,-79.265758
3668,13456297,Danforth Rd / North Edgely Ave,43.709715,-79.265758
3669,13441958,Zoo Rd / Kirkhams Rd / Kirkhams Rd Zoo Rd Ramp,43.818247,-79.175723


In [21]:
stops = stops.drop_duplicates()

In [22]:
stop_times = stop_times[stop_times['trip_id'].isin(list(trips['trip_id']))]

In [23]:
agency = pd.read_csv(gtfs_dir + 'agency.txt')
calendar = pd.read_csv(gtfs_dir + 'calendar.txt')
calendar_dates = pd.read_csv(gtfs_dir + 'calendar_dates.txt')

In [24]:
feed_table = [['TTC', 'https://ttc.ca', 'en', '20160101', '20170101', '2016-08', 'ttc']]

feed_info = pd.DataFrame.from_records(feed_table, columns = ['feed_publisher_name','feed_publisher_url',
                                                             'feed_lang','feed_start_date','feed_end_date',
                                                             'feed_version','feed_id'])

In [25]:
os.makedirs(gtfs_dir + 'ttc_otp', exist_ok = True)

In [26]:
stops.to_csv(gtfs_dir + 'ttc_otp/stops.txt', index = False)
routes.to_csv(gtfs_dir + 'ttc_otp/routes.txt', index = False)
stop_times.to_csv(gtfs_dir + 'ttc_otp/stop_times.txt', index = False)
trips.to_csv(gtfs_dir + 'ttc_otp/trips.txt', index = False)
agency.to_csv(gtfs_dir + 'ttc_otp/agency.txt', index = False)
calendar.to_csv(gtfs_dir + 'ttc_otp/calendar.txt', index = False)
calendar_dates.to_csv(gtfs_dir + 'ttc_otp/calendar_dates.txt', index = False)
feed_info.to_csv(gtfs_dir + 'ttc_otp/feed_info.txt', index = False)

In [27]:
shutil.make_archive(gtfs_dir + 'ttc_otp_weekday.gtfs', 'zip', gtfs_dir + 'ttc_otp' )

'/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/ttc_otp_weekday.gtfs.zip'